In [73]:
import pandas as pd
import numpy as np

In [74]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [75]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data["Гормональная активность 0-нет               1-да"].fillna("missing")

In [76]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
data.dropna(axis=0, thresh=int(data.shape[1]*0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0]*0.6), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)

In [77]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

In [79]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(loss_function = "Logloss",
                           cat_features = ["Гормональная активность 0-нет               1-да"],
                           verbose=5,
                           thread_count=-1,
                           random_state=42)

In [80]:
"""from sklearn.model_selection import GridSearchCV

param_grid = {
    'iterations': [20, 40, 60],
    'learning_rate': [0.01, 0.1, 0.5],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)"""

"from sklearn.model_selection import GridSearchCV\n\nparam_grid = {\n    'iterations': [20, 40, 60],\n    'learning_rate': [0.01, 0.1, 0.5],\n    'depth': [6, 8, 10],\n    'l2_leaf_reg': [1, 3, 5]\n}\n\ngrid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)\n\ngrid_search.fit(X_train, y_train)\nprint(grid_search.best_params_)"

In [81]:
best_model = CatBoostClassifier(iterations=20,
                                depth=10,
                                l2_leaf_reg=3,
                                learning_rate=0.1,
                                loss_function = "Logloss",
                                verbose=5,
                                thread_count=-1,
                                random_state=42)

In [82]:
"""start_idx = 0
end_idx = int(len(X)*0.8)
step = 2
while end_idx <= X.shape[0]:
    current_training_data = (X[start_idx:end_idx], y[start_idx:end_idx])
    if start_idx == 0:
        current_validating_data = (X[end_idx:], y[end_idx:])
    else:
        current_validating_data = (np.concatenate((X[end_idx:], X[:start_idx])),  np.concatenate((y[end_idx:], y[:start_idx])))
    if start_idx == 0 :
        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True)
    else:
        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True,
                       init_model=best_model)
    start_idx += step
    end_idx += step"""

'start_idx = 0\nend_idx = int(len(X)*0.8)\nstep = 2\nwhile end_idx <= X.shape[0]:\n    current_training_data = (X[start_idx:end_idx], y[start_idx:end_idx])\n    if start_idx == 0:\n        current_validating_data = (X[end_idx:], y[end_idx:])\n    else:\n        current_validating_data = (np.concatenate((X[end_idx:], X[:start_idx])),  np.concatenate((y[end_idx:], y[:start_idx])))\n    if start_idx == 0 :\n        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True)\n    else:\n        best_model.fit(current_training_data[0], current_training_data[1], eval_set=current_validating_data, plot=True,\n                       init_model=best_model)\n    start_idx += step\n    end_idx += step'

In [83]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {i}:")
    xtrain, xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    if i == 0:
        best_model.fit(xtrain, ytrain, eval_set=(xtest, ytest), plot=True)
    else:
        best_model.fit(xtrain, ytrain, eval_set=(xtest, ytest), init_model=best_model, plot=True)

Fold 0:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6249053	test: 0.6641286	best: 0.6641286 (0)	total: 11.5ms	remaining: 218ms
5:	learn: 0.3791852	test: 0.5385835	best: 0.5385835 (5)	total: 58.1ms	remaining: 136ms
10:	learn: 0.2585224	test: 0.4661201	best: 0.4661201 (10)	total: 114ms	remaining: 93.6ms
15:	learn: 0.1725156	test: 0.3973093	best: 0.3973093 (15)	total: 155ms	remaining: 38.8ms
19:	learn: 0.1412074	test: 0.3853474	best: 0.3853474 (19)	total: 195ms	remaining: 0us

bestTest = 0.3853474003
bestIteration = 19

Fold 1:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.1931893	test: 0.1321731	best: 0.1321731 (0)	total: 11.7ms	remaining: 222ms
5:	learn: 0.1450706	test: 0.1247712	best: 0.1247712 (5)	total: 59.1ms	remaining: 138ms
10:	learn: 0.1127551	test: 0.1192765	best: 0.1192765 (10)	total: 105ms	remaining: 86ms
15:	learn: 0.0890999	test: 0.1067913	best: 0.1067913 (15)	total: 142ms	remaining: 35.6ms
19:	learn: 0.0775998	test: 0.1029600	best: 0.1029600 (19)	total: 184ms	remaining: 0us

bestTest = 0.1029599537
bestIteration = 19

Fold 2:


...

0:	learn: 0.0836869	test: 0.0628278	best: 0.0628278 (0)	total: 10.2ms	remaining: 194ms
5:	learn: 0.0675051	test: 0.0553164	best: 0.0553164 (5)	total: 57.2ms	remaining: 134ms
10:	learn: 0.0587698	test: 0.0533798	best: 0.0533669 (9)	total: 109ms	remaining: 89.2ms
15:	learn: 0.0519843	test: 0.0516609	best: 0.0516609 (15)	total: 160ms	remaining: 39.9ms
19:	learn: 0.0458455	test: 0.0490854	best: 0.0490854 (19)	total: 177ms	remaining: 0us

bestTest = 0.0490853575
bestIteration = 19

Fold 3:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0470298	test: 0.0394107	best: 0.0394107 (0)	total: 9.74ms	remaining: 185ms
5:	learn: 0.0420347	test: 0.0384968	best: 0.0384968 (5)	total: 63.3ms	remaining: 148ms
10:	learn: 0.0380075	test: 0.0374548	best: 0.0374548 (10)	total: 111ms	remaining: 90.5ms
15:	learn: 0.0347762	test: 0.0370981	best: 0.0370981 (15)	total: 156ms	remaining: 39.1ms
19:	learn: 0.0324823	test: 0.0370023	best: 0.0370023 (19)	total: 193ms	remaining: 0us

bestTest = 0.03700225097
bestIteration = 19

Fold 4:


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0346065	test: 0.0259321	best: 0.0259321 (0)	total: 10.7ms	remaining: 203ms
5:	learn: 0.0319258	test: 0.0253478	best: 0.0253478 (5)	total: 61.5ms	remaining: 144ms
10:	learn: 0.0295098	test: 0.0245592	best: 0.0245592 (10)	total: 112ms	remaining: 91.7ms
15:	learn: 0.0271786	test: 0.0236681	best: 0.0236681 (15)	total: 154ms	remaining: 38.5ms
19:	learn: 0.0257024	test: 0.0232142	best: 0.0232142 (19)	total: 192ms	remaining: 0us

bestTest = 0.02321416215
bestIteration = 19



In [84]:
from sklearn.model_selection import cross_val_score

cv = cross_val_score(best_model, X, y, n_jobs=-1)
cv

array([1.       , 0.9047619, 0.95     , 0.9      , 0.85     ])

In [85]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.86      0.92         7

    accuracy                           0.91        11
   macro avg       0.90      0.93      0.91        11
weighted avg       0.93      0.91      0.91        11



In [86]:
feature_importances = best_model.get_feature_importance()
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
3,Weiss,34.436076
1,"Максимальный размер, мм",8.162881
28,16dP2_3A,4.460840
51,aTHB_THB,3.304448
54,THE_11oxoP3,2.851907
29,dA3,2.786565
48,THF_THE1,1.883777
52,11An_11Et,1.641268
14,Et,1.615940
49,THF_THE,1.582744
